In [1]:
import pandas as pd
from random import sample

In [2]:
import numpy as np
import category_encoders as ce

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

In [4]:
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier

In [6]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
from sklearn.neighbors import KNeighborsClassifier

In [7]:
from sklearn.model_selection import KFold

## Train y predict set

In [8]:
features_04 = pd.read_csv('features_06.csv')
features_04 = features_04.drop(['Unknown','Unknown.1'],axis=1)
labels = pd.read_csv('labels_training_set.csv')
kaggle = pd.read_csv('trocafone_kaggle_test.csv')

In [11]:
training = labels.set_index('person').join(features_04.set_index('person'),how='left')

In [12]:
training_set = training.drop('label',axis=1)
training_labels = training['label']

In [14]:
prediction_set = kaggle.set_index('person').join(features_04.set_index('person'),how='left')

In [15]:
cat = []
catnames = []
for index,v in enumerate(training_set.columns):
    if training_set[v].dtype.name == 'object':
        cat.append(index)
        catnames.append(v)

In [16]:
for i in catnames:
    training_set[i] = training_set[i].astype('category')
    prediction_set[i] = prediction_set[i].astype('category')

## Funciones

In [17]:
def agregarFeatureTrain (dataset,labels,clasif_list,n):
    to_return = []
    new_dataset = dataset.copy(deep=True)
    kf = KFold(n_splits=n)
    new_feature = []
    for clasificador in clasif_list:
        for a,b in kf.split(dataset):
            clasificador.fit(dataset.iloc[a],labels.iloc[a])
            new_labels = clasificador.predict_proba(dataset.iloc[b])[:,1]
            new_feature.extend(new_labels)
        newindex = len(new_dataset.columns)+1
        new_dataset[newindex] = new_feature
        to_return.append(newindex)
    return new_dataset.loc[:,to_return]

In [18]:
def agregarFeatureTrainCB (dataset,labels,clasif_list,n):
    to_return = []
    new_dataset = dataset.copy(deep=True)
    kf = KFold(n_splits=n)
    new_feature = []
    for clasificador in clasif_list:
        for a,b in kf.split(dataset):
            clasificador.fit(dataset.iloc[a],labels.iloc[a],cat_features=cat)
            new_labels = clasificador.predict_proba(dataset.iloc[b])[:,1]
            new_feature.extend(new_labels)
        newindex = len(new_dataset.columns)+1
        new_dataset[newindex] = new_feature
        to_return.append(newindex)
    return new_dataset.loc[:,to_return]

In [19]:
def agregarFeaturePredict(dataset,labels,predict,clasif_list):
    to_return = []
    new_predict = predict.copy(deep=True)
    for classifier in clasif_list:
        classifier.fit(dataset,labels)
        prediction = classifier.predict_proba(predict)[:,1]
        newindex = len(predict.columns)+1
        new_predict[newindex] = prediction
        to_return.append(newindex)
    return new_predict.loc[:,to_return]

In [20]:
def agregarFeaturePredictCB(dataset,labels,predict,clasif_list):
    to_return = []
    new_predict = predict.copy(deep=True)
    for classifier in clasif_list:
        classifier.fit(dataset,labels,cat_features=cat)
        prediction = classifier.predict_proba(predict)[:,1]
        newindex = len(predict.columns)+1
        new_predict[newindex] = prediction
        to_return.append(newindex)
    return new_predict.loc[:,to_return]

## Clasificadores

In [21]:

lgbmparams = {   }
cb_classifier = cb.CatBoostClassifier(iterations=400,
                                 depth=10,
                                 l2_leaf_reg=50,
                                 random_strength=0.1,
                                 bagging_temperature=0,
                                 border_count=128)
xgb_classifier = xgb.XGBClassifier(colsample_bytree = 0.606653667095343,
             gamma = 7,
             learning_rate = 0.016238752458245277,
             max_depth = 12,
             min_child_weight = 5,
             n_estimators = 346,
             subsample = 0.7847190225361189)
lgbm_classifier = lgb.LGBMClassifier(learning_rate = 0.06748371705664348,
                 max_bin = 205,
                 min_data_in_leaf = 481,
                 n_estimators = 187,
                 num_leaves = 956)

## Encoding (para xgboost)

In [22]:
enc = ce.TargetEncoder()
encoded = enc.fit_transform(np.array(training_set),np.array(training_labels))
encoded_p = enc.transform(np.array(prediction_set))

## Obtener features

In [23]:
train_set1 = agregarFeatureTrain(encoded,training_labels,[xgb_classifier],4)
predict_set1 = agregarFeaturePredict(encoded,training_labels,encoded_p,[xgb_classifier])

In [24]:
train_set2 = agregarFeatureTrain(training_set,training_labels,[lgbm_classifier],4)
predict_set2 = agregarFeaturePredict(training_set,training_labels,prediction_set,[lgbm_classifier])

In [ ]:
train_set3 = agregarFeatureTrainCB(training_set,training_labels,[cb_classifier],4)
predict_set3 = agregarFeaturePredictCB(training_set,training_labels,prediction_set,[cb_classifier])

0:	learn: 0.6491524	total: 227ms	remaining: 1m 30s
1:	learn: 0.6080217	total: 375ms	remaining: 1m 14s
2:	learn: 0.5705140	total: 562ms	remaining: 1m 14s
3:	learn: 0.5344591	total: 699ms	remaining: 1m 9s
4:	learn: 0.4999938	total: 1.04s	remaining: 1m 21s
5:	learn: 0.4694659	total: 1.21s	remaining: 1m 19s
6:	learn: 0.4445654	total: 1.38s	remaining: 1m 17s
7:	learn: 0.4211542	total: 1.55s	remaining: 1m 16s
8:	learn: 0.4031227	total: 1.75s	remaining: 1m 16s
9:	learn: 0.3810654	total: 1.88s	remaining: 1m 13s
10:	learn: 0.3662000	total: 2.06s	remaining: 1m 12s
11:	learn: 0.3520199	total: 2.91s	remaining: 1m 34s
12:	learn: 0.3365371	total: 3.51s	remaining: 1m 44s
13:	learn: 0.3247350	total: 3.65s	remaining: 1m 40s
14:	learn: 0.3116942	total: 3.87s	remaining: 1m 39s
15:	learn: 0.3017635	total: 4.79s	remaining: 1m 55s
16:	learn: 0.2927304	total: 5.79s	remaining: 2m 10s
17:	learn: 0.2818860	total: 6.18s	remaining: 2m 11s
18:	learn: 0.2739596	total: 6.31s	remaining: 2m 6s
19:	learn: 0.2648946	tot

In [193]:
train_set2.columns = ['lgbm']
train_set3.columns = ['catboost']

In [194]:
predict_set2.columns = ['lgbm']
predict_set3.columns = ['catboost']

## Armar stack (training)

In [197]:
stacked_training = train_set2.join(train_set3)
stacked_training['xgboost'] = list(train_set1[46])

In [198]:
stacked_training.head()

,lgbm,catboost,xgboost
person,,,
0566e9c1,0.031517,0.044850,0.056806
6ec7ee77,0.048258,0.062670,0.068039
abe7a2fb,0.046011,0.023425,0.031733
34728364,0.312454,0.156923,0.341176
87ed62de,0.017839,0.011140,0.012972


## Armar stack (prediction)

In [199]:
stacked_predict = predict_set2.join(predict_set3)
stacked_predict['xgboost'] = list(predict_set1[46])

In [200]:
stacked_predict.head()

,lgbm,catboost,xgboost
person,,,
4886f805,0.001365,0.007907,0.007319
0297fc1e,0.012040,0.060722,0.032214
2d681dd8,0.006546,0.016244,0.012345
cccea85e,0.035064,0.052161,0.074387
4c8a8b93,0.003504,0.024271,0.024076


## Prediccion final (Score: 0.85527)

##### Labels de xgboost, catboost y lightgbm con Logistic Regression como modelo final

In [201]:
log = LogisticRegression()

In [346]:
stacked_training.to_csv('preentrenados/training_logrec')

In [347]:
stacked_predict.to_csv('preentrenados/predict_logrec')

In [367]:
xtrain,xtest,ytrain,ytest = train_test_split(stacked_training,labels['label'])

In [368]:
log.fit(xtrain,ytrain)
roc_auc_score(ytest,log.predict_proba(xtest)[:,1])

0.8627505908628511

In [369]:
log.fit(stacked_training,labels['label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [370]:
final_pred = log.predict_proba(stacked_predict)[:,1]

In [371]:
submit = kaggle.copy(deep=True)
submit['label'] = final_pred

## Usando algunas features del dataset 

In [307]:
fts = ['busco_productos', 'clickeo_ad', 'hizo_checkout', 'entro_desde_motor',
       'hizo_conversion', 'filtro_busquedas', 'busco_marca', 'visito_sitio',
       'hizo_lead', 'modelos_distintos', 'busquedas',
       'clicks_en_ads', 'checkouts', 'entradas_desde_motor',
       'listar_productos', 'listar_marca']

In [308]:
train_f = stacked_training.join(training_set.loc[:,fts])
pred_f = stacked_predict.join(prediction_set.loc[:,fts])

In [319]:
xtrain,xtest,ytrain,ytest = train_test_split(train_f,labels['label'])

In [320]:
log.fit(xtrain,ytrain)
roc_auc_score(ytest,log.predict_proba(xtest)[:,1])

0.8584317609192725

In [214]:
log.fit(train_f,labels['label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [215]:
n_prediction = log.predict_proba(pred_f)[:,1]

In [216]:
submit = kaggle.set_index('person').copy(deep=True)
submit['label'] = n_prediction

## Xgboost con todas las features

In [322]:
forxgboost = stacked_training.reset_index().join(encoded).set_index('person')

In [323]:
xtrain,xtest,ytrain,ytest = train_test_split(forxgboost,labels['label'])

In [324]:
xgb_classifier.fit(xtrain,ytrain)
roc_auc_score(ytest,xgb_classifier.predict_proba(xtest)[:,1])

0.8638913423827671

## Agrego KNN y RandomForests

##### KNN

In [222]:
from sklearn.preprocessing import normalize

In [228]:
normalized = pd.DataFrame(normalize(encoded,axis=0))
normalized_p = pd.DataFrame(normalize(encoded_p,axis=0))

In [229]:
knn = KNeighborsClassifier(leaf_size=15,p=3)

##### RandomForest

In [230]:
train_set_knn = agregarFeatureTrain(normalized,training_labels,[knn],4)
predict_set_knn = agregarFeaturePredict(normalized,training_labels,normalized_p,[knn])

In [233]:
rf = RandomForestClassifier(n_estimators=100)

In [234]:
train_set_rf = agregarFeatureTrain(encoded,training_labels,[rf],4)
predict_set_rf = agregarFeaturePredict(encoded,training_labels,encoded_p,[rf])

#### Armo set

In [244]:
stacked_features = train_set2.join(train_set3,how='left')
stacked_features['knn'] = list(train_set_knn[46])
stacked_features['rf'] = list(train_set_rf[46])
stacked_features['xgboost'] = list(train_set1[46])

In [245]:
stacked_features_p = predict_set2.join(predict_set3,how='left')
stacked_features_p['knn'] = list(predict_set_knn[46])
stacked_features_p['rf'] = list(predict_set_rf[46])
stacked_features_p['xgboost'] = list(predict_set1[46])

## Prediccion

In [268]:
log = LogisticRegression()

In [361]:
xtrain,xtest,ytrain,ytest = train_test_split(stacked_features,labels['label'],train_size=0.9,shuffle=True)

C:\Users\javif\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [362]:
log.fit(xtrain,ytrain)
roc_auc_score(ytest,log.predict_proba(xtest)[:,1])

0.860789117435889

In [352]:
log.fit(stacked_features,labels['label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [353]:
final_pred = log.predict_proba(stacked_features_p)[:,1]

In [354]:
submit = kaggle.copy(deep=True)
submit['label'] = final_pred

## XGBoost con de todo

In [374]:
import scipy.stats as stats

In [376]:
from sklearn.model_selection import RandomizedSearchCV

In [378]:
params={
    'n_estimators':stats.randint(150,500),
    'learning_rate':stats.uniform(0.01,0.3),
    'subsample':stats.uniform(0.3,0.7),
    'min_child_weight':[1,5,10],
    'max_depth':[3,10,6],
    'gamma':stats.randint(0,10),
    'colsample_bytree':stats.uniform(0.4,0.6)
}
grid = RandomizedSearchCV(xgb.XGBClassifier(n_jobs=-1),
                          param_distributions=params,
                          scoring='roc_auc',
                          cv=2,
                          verbose=1,
                          n_iter=100)

In [379]:
forxgboost2 = stacked_features.reset_index().join(encoded).set_index('person')

In [380]:
xgboost2predict = stacked_features_p.reset_index().join(encoded_p).set_index('person')

In [381]:
xtrain,xtest,ytrain,ytest = train_test_split(forxgboost2,labels['label'])

In [382]:
grid.fit(xtrain,ytrain)
roc_auc_score(ytest,grid.predict_proba(xtest)[:,1])

Fitting 2 folds for each of 100 candidates, totalling 200 fits


[Parallel(n_jobs=1)]: Done 200 out of 200 | elapsed: 27.2min finished


0.8609554080566848

In [383]:
grid.best_estimator_.fit(forxgboost2,labels['label'])

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5507688588435546, gamma=8,
       learning_rate=0.02746639067303843, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=181, n_jobs=-1,
       nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=0.838043777420667)

In [384]:
submit = kaggle.copy(deep=True).set_index('person')
submit['label'] = grid.best_estimator_.predict_proba(xgboost2predict)[:,1]

## Logistic regression tuneando parametros (score: 0.85527)

In [421]:
logreg = LogisticRegression()
params = {
    'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
}
log = RandomizedSearchCV(logreg,param_distributions=params,verbose=0,cv=4,n_iter=5)

In [346]:
stacked_training.to_csv('preentrenados/training_logrec')

In [347]:
stacked_predict.to_csv('preentrenados/predict_logrec')

In [422]:
xtrain,xtest,ytrain,ytest = train_test_split(stacked_training,labels['label'])

In [423]:
log.fit(xtrain,ytrain)
roc_auc_score(ytest,log.predict_proba(xtest)[:,1])

0.8681777303642428

In [424]:
log.best_estimator_.fit(stacked_training,labels['label'])

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [415]:
final_pred = log.best_estimator_.predict_proba(stacked_predict)[:,1]

In [416]:
submit = kaggle.copy(deep=True)
submit['label'] = final_pred